In [1]:
# Step 1: Import required libraries
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score

# Step 2: Load the Titanic dataset
df = sns.load_dataset('titanic')

# Step 3: Drop unnecessary or mostly-empty columns
df = df.drop(columns=['deck', 'embark_town', 'alive'])

# Step 4: Handle missing values
df['age'] = df['age'].fillna(df['age'].mean())   # ✅ Fix: no inplace=True
df = df.dropna(subset=['embarked'])              # Drop rows where 'embarked' is missing
df = df.dropna()                                 # Drop any remaining missing data

# Step 5: Convert categorical columns to numeric using LabelEncoder
label_encoder = LabelEncoder()
for col in df.columns:
    if df[col].dtype == 'object' or str(df[col].dtype) == 'category':
        df[col] = label_encoder.fit_transform(df[col].astype(str))  # ✅ Fix: convert to str

# Step 6: Normalize numerical columns using MinMaxScaler
scaler = MinMaxScaler()
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Step 7: Split the data into features (X) and target (y)
X = df.drop('survived', axis=1)  # Features
y = df['survived']               # Target

# Step 8: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 9: Train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Step 10: Predict and evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("\n✅ Model Accuracy: {:.2f}%".format(accuracy * 100))



✅ Model Accuracy: 80.34%


In [16]:
# Step 1: Import necessary libraries
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Step 2: Load dataset
df = sns.load_dataset('titanic')

# Step 3: Drop unnecessary or mostly empty columns
df = df.drop(columns=['deck', 'embark_town', 'alive'])

# Step 4: Handle missing values
df['age'] = df['age'].fillna(df['age'].mean())
df = df.dropna(subset=['embarked'])
df = df.dropna()

# Step 5: Encode categorical variables
label_encoder = LabelEncoder()
for col in df.columns:
    if df[col].dtype == 'object' or str(df[col].dtype) == 'category':
        df[col] = label_encoder.fit_transform(df[col].astype(str))

# Step 6: Normalize numerical columns
scaler = MinMaxScaler()
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Step 7: Split into features and target
X = df.drop('survived', axis=1)
y = df['survived']

# Step 8: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 9: Train Decision Tree Classifier
tree_model = DecisionTreeClassifier(max_depth=4, random_state=42)  # Limit depth for clear visualization
tree_model.fit(X_train, y_train)

# Step 10: Evaluate model
y_pred = tree_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\n Decision Tree Accuracy: {accuracy:.2f}")

## Step 11: Visualize the tree
#plt.figure(figsize=(20, 10))
#plot_tree(tree_model, feature_names=X.columns, class_names=['Not Survived', 'Survived'],
 #         filled=True, rounded=True, fontsize=12)
#plt.title("Decision Tree for Titanic Survival")
#plt.show()

# Step 12: Predict on a new sample (e.g. hypothetical passenger)
# NOTE: Input values must match the order and scaling of your features
# Example: [pclass, sex, age, sibsp, parch, fare, embarked, class, who, adult_male, alone]
# We'll use the first row from the training set as an example of a "new passenger"
# Fix: Create a new DataFrame with correct feature names
sample = pd.DataFrame([X.iloc[0].values], columns=X.columns)
prediction = tree_model.predict(sample)#

print("\n Prediction for new sample:", "Survived" if prediction[0] == 1 else "Not Survived")




 Decision Tree Accuracy: 0.81

 Prediction for new sample: Not Survived


In [29]:
import pandas as pd
import seaborn as sns
from sklearn.datasets import load_iris, load_wine, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# ✅ Updated preprocessing function
def preprocess(df, target_column):
    df = df.copy()  # prevent chained assignment
    df = df.dropna()

    # Label encode object and category columns
    for col in df.columns:
        if df[col].dtype == 'object' or str(df[col].dtype) == 'category':
            df[col] = LabelEncoder().fit_transform(df[col].astype(str))

    # Separate features and target
    X = df.drop(target_column, axis=1)
    y = df[target_column]

    # Scale numerical features
    scaler = MinMaxScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

    return train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Decision tree pruning strategies
pruning_params = [
    {'max_depth': 3},
    {'max_depth': 5},
    {'min_samples_split': 10},
    {'min_samples_leaf': 5},
    {'max_leaf_nodes': 10}
]

# Datasets: (name, df, target_column)
datasets = []

# 1. Titanic
titanic = sns.load_dataset('titanic')
titanic = titanic.drop(columns=['deck', 'embark_town', 'alive'])
datasets.append(('Titanic', titanic, 'survived'))

# 2. Iris
iris = load_iris(as_frame=True)
datasets.append(('Iris', iris.frame, 'target'))

# 3. Wine
wine = load_wine(as_frame=True)
datasets.append(('Wine', wine.frame, 'target'))

# 4. Breast Cancer
cancer = load_breast_cancer(as_frame=True)
datasets.append(('Breast Cancer', cancer.frame, 'target'))

# 5. Penguins
penguins = sns.load_dataset('penguins')
penguins = penguins.drop(columns=['island', 'species'])
penguins = penguins[penguins['sex'].notna()]  # drop missing targets
penguins['sex'] = penguins['sex'].map({'Male': 1, 'Female': 0})
datasets.append(('Penguins', penguins, 'sex'))

# Main loop to collect results
results = []

for dataset_name, df, target in datasets:
    try:
        X_train, X_test, y_train, y_test = preprocess(df, target)

        for params in pruning_params:
            clf = DecisionTreeClassifier(**params, random_state=42)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            acc = accuracy_score(y_test, y_pred)

            results.append({
                'Dataset': dataset_name,
                'Pruning Method': str(params),
                'Accuracy': round(acc * 100, 2)
            })

    except Exception as e:
        print(f"❌ Error with dataset {dataset_name}: {e}")

# Create summary table
results_df = pd.DataFrame(results)

# Pivot for comparison
comparison_table = results_df.pivot(index='Dataset', columns='Pruning Method', values='Accuracy')
print("\n Decision Tree Pruning Comparison:\n")
print(comparison_table.round(2))


 Decision Tree Pruning Comparison:

Pruning Method  {'max_depth': 3}  {'max_depth': 5}  {'max_leaf_nodes': 10}  \
Dataset                                                                      
Breast Cancer              94.74             94.74                   94.74   
Iris                      100.00            100.00                  100.00   
Penguins                   83.58             86.57                   83.58   
Titanic                    76.92             70.63                   74.83   
Wine                       94.44             94.44                   94.44   

Pruning Method  {'min_samples_leaf': 5}  {'min_samples_split': 10}  
Dataset                                                             
Breast Cancer                     95.61                      94.74  
Iris                             100.00                     100.00  
Penguins                          85.07                      85.07  
Titanic                           79.02                      72.03  
Wi